# Execute on different backends

In [1]:
from IPython.display import HTML, display

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
import featuremesh
%reload_ext featuremesh

from featuremesh import generate_access_token, decode_token

from libs.helpers.utils import get_featuremesh_config

fm_config = get_featuremesh_config()

featuremesh.set_default('registry.host', fm_config['registry.host'])
featuremesh.set_default('access.host', fm_config['access.host'])

__YOUR_IDENTITY_TOKEN__ = fm_config['identity_token']
__YOUR_ACCESS_TOKEN__ = generate_access_token(identity_token=__YOUR_IDENTITY_TOKEN__, project='default')

# decode_token(__YOUR_ACCESS_TOKEN__)

In [3]:
# Create clients

import datafusion
from libs.helpers.utils_db import query_trino, query_duckdb, query_bigquery, query_datafusion

client_duckdb_anon = featuremesh.OfflineClient(None, featuremesh.Backend.DUCKDB, query_duckdb)
client_duckdb = featuremesh.OfflineClient(__YOUR_ACCESS_TOKEN__, featuremesh.Backend.DUCKDB, query_duckdb)
client_trino = featuremesh.OfflineClient(__YOUR_ACCESS_TOKEN__, featuremesh.Backend.TRINO, query_trino)
client_bigquery = featuremesh.OfflineClient(__YOUR_ACCESS_TOKEN__, featuremesh.Backend.BIGQUERY, query_bigquery)
client_datafusion = featuremesh.OfflineClient(__YOUR_ACCESS_TOKEN__, featuremesh.Backend.DATAFUSION, query_datafusion)

client_online = featuremesh.OnlineClient(access_token=__YOUR_ACCESS_TOKEN__)

featuremesh.set_default('client', client_duckdb)

## Check executors run fine

In [4]:
# import os
# import signal
# os.kill(63768, signal.SIGTERM)

In [5]:
# Login to BigQuery using: 
# gcloud auth application-default login --quiet --project=your_project
# or
# gcloud auth application-default login
# gcloud auth application-default set-quota-project your_project

In [6]:
# !pip install datafusion==48.0.0

In [7]:
datafusion.__version__

'51.0.0'

In [8]:
query = "SELECT 1+2 AS COL1"
for sql_executor in [query_duckdb, query_trino, query_bigquery, query_datafusion]:
    display(HTML(f"<h3>Execute query with {sql_executor.__name__}</h3>"))
    display(sql_executor(query))

,COL1
0,3


,COL1
0,3


,COL1
0,3


,col1
0,3


## Execute with local client and magic commands

In [9]:
%%featureql

WITH 
    INPUT(BIGINT) AS INPUT1, 
    INPUT(BIGINT) AS INPUT2
SELECT 
    BIND_VALUES(ARRAY[ROW(2,3), ROW(2,4), ROW(2,5)]) AS (INPUT1, INPUT2), 
    INPUT1 + INPUT2 AS RESULT
;

,INPUT1,INPUT2,RESULT
0,2,3,5
1,2,4,6
2,2,5,7


## Execute on 4+1 different backends with local client

In [10]:
query = """
    WITH 
        INPUT(BIGINT) AS INPUT1, 
        INPUT(BIGINT) AS INPUT2
    SELECT 
        BIND_VALUES(ARRAY[ROW(2,3), ROW(2,4), ROW(2,5)]) AS (INPUT1, INPUT2), 
        INPUT1 + INPUT2 AS RESULT
    ;
"""

for client in [client_duckdb, client_trino, client_bigquery, client_datafusion]: 
    display(HTML(f"<h3>Execute query with {client.backend}</h3>"))
    try:
        response = client.query(query)
        print(response.sql)
        print(response.dataframe)
    except Exception as e:
        print(f'A problem happened {e=}')

display(HTML(f"<h3>Execute query online</h3>"))
try:
    response = client_online.query(query)
    print(response.sql)
    print(response.dataframe)
except:
    print('A problem happened')

/* START_OF_QUERY reference: default */
WITH
CTE_324PPV6H_0 AS (
            (
                SELECT t['field_1'] as INPUT1, t['field_2'] as INPUT2 FROM (SELECT UNNEST(
            CAST(ARRAY[STRUCT_PACK("field_1" := 2, "field_2" := 3), STRUCT_PACK("field_1" := 2, "field_2" := 4), STRUCT_PACK("field_1" := 2, "field_2" := 5)] AS ROW("field_1" BIGINT, "field_2" BIGINT)[])
        ) as t)
        )
), 
CTE_324PPV6H_1 AS (
SELECT
	CAST(INPUT1 AS BIGINT) AS INPUT1, 
	CAST(INPUT2 AS BIGINT) AS INPUT2
FROM CTE_324PPV6H_0
), 
CTE_324PPV6H_2 AS (
SELECT
	(INPUT1 + INPUT2) AS RESULT, 
	INPUT1, 
	INPUT2
FROM CTE_324PPV6H_1
)
/*
reference = default [final layer=3]
CteTypeEnum.FINAL
*/
SELECT
    INPUT1, INPUT2, RESULT
FROM CTE_324PPV6H_2
/* END_OF_QUERY reference: default */

   INPUT1  INPUT2  RESULT
0       2       3       5
1       2       4       6
2       2       5       7


/* START_OF_QUERY reference: default */
WITH
CTE_324PPV6H_0 AS (
            (
            SELECT t1.*
            FROM UNNEST(
            CAST(ARRAY[CAST(ROW(2, 3) AS ROW(field_1 BIGINT, field_2 BIGINT)), CAST(ROW(2, 4) AS ROW(field_1 BIGINT, field_2 BIGINT)), CAST(ROW(2, 5) AS ROW(field_1 BIGINT, field_2 BIGINT))] AS ARRAY(ROW(field_1 BIGINT, field_2 BIGINT)))
        )
            AS t1(INPUT1, INPUT2)
        )
), 
CTE_324PPV6H_1 AS (
SELECT
	CAST(INPUT1 AS BIGINT) AS INPUT1, 
	CAST(INPUT2 AS BIGINT) AS INPUT2
FROM CTE_324PPV6H_0
), 
CTE_324PPV6H_2 AS (
SELECT
	(INPUT1 + INPUT2) AS RESULT, 
	INPUT1, 
	INPUT2
FROM CTE_324PPV6H_1
)
/*
reference = default [final layer=3]
CteTypeEnum.FINAL
*/
SELECT
    INPUT1, INPUT2, RESULT
FROM CTE_324PPV6H_2
/* END_OF_QUERY reference: default */

   INPUT1  INPUT2  RESULT
0       2       3       5
1       2       4       6
2       2       5       7


/* START_OF_QUERY reference: default */
WITH
CTE_324PPV6H_0 AS (
            (
                SELECT field_1 as INPUT1, field_2 as INPUT2 FROM (SELECT * FROM UNNEST(
            CAST(ARRAY[CAST(STRUCT(2, 3) AS STRUCT<field_1 INT64, field_2 INT64>), CAST(STRUCT(2, 4) AS STRUCT<field_1 INT64, field_2 INT64>), CAST(STRUCT(2, 5) AS STRUCT<field_1 INT64, field_2 INT64>)] AS ARRAY<STRUCT<field_1 INT64, field_2 INT64>>)
        ))
        )
), 
CTE_324PPV6H_1 AS (
SELECT
	CAST(INPUT1 AS INT64) AS INPUT1, 
	CAST(INPUT2 AS INT64) AS INPUT2
FROM CTE_324PPV6H_0
), 
CTE_324PPV6H_2 AS (
SELECT
	(INPUT1 + INPUT2) AS RESULT, 
	INPUT1, 
	INPUT2
FROM CTE_324PPV6H_1
)
/*
reference = default [final layer=3]
CteTypeEnum.FINAL
*/
SELECT
    INPUT1, INPUT2, RESULT
FROM CTE_324PPV6H_2
/* END_OF_QUERY reference: default */

   INPUT1  INPUT2  RESULT
0       2       3       5
1       2       4       6
2       2       5       7


/* start_of_query reference: default */
with
cte_324ppv6h_0 as (
            (
            select t1.*
            from 
                unnest(unnest(make_array(named_struct('field_1', 2, 'field_2', 3), named_struct('field_1', 2, 'field_2', 4), named_struct('field_1', 2, 'field_2', 5))))
            as t1(input1, input2)
        )
), 
cte_324ppv6h_1 as (
select
	cast(input1 as bigint) as input1, 
	cast(input2 as bigint) as input2
from cte_324ppv6h_0
), 
cte_324ppv6h_2 as (
select
	(input1 + input2) as result, 
	input1, 
	input2
from cte_324ppv6h_1
)
/*
reference = default [final layer=3]
ctetypeenum.final
*/
select
    input1, input2, result
from cte_324ppv6h_2
/* end_of_query reference: default */

   input1  input2  result
0       2       3       5
1       2       4       6
2       2       5       7


None
   INPUT1  INPUT2  RESULT
0       2       3       5
1       2       4       6
2       2       5       7
